# Data Transforms
---

There are many pre-built transforms available -
- To PIL Image.
- To Tensors.
- Generic - Using `Lambda` functions.
- Functional Transforms.
- Chain together multiple transforms using the `Compose` method.
..

In [7]:
import torch
from torch.utils.data import Dataset
import torchvision
import numpy as np

In [19]:
class WineDataset(Dataset):
    def __init__(self, transform=None):
        # Data loading
        xy = np.loadtxt('wine.csv', delimiter=",", dtype=np.float32, skiprows=1)
        self.x = xy[:, 1:]
        self.y = xy[:, [0]]
        self.n_samples = xy.shape[0]
        self.transform = transform
    def __getitem__(self, index):
        # return the item at a specific index
        # Apply Transform if available
        sample = self.x[index], self.y[index]
        if self.transform:
            sample = self.transform(sample)
        return sample
    def __len__(self):
        # Length of the dataset
        return self.n_samples

In [16]:
class ToTensor():
    """
    Custom To Tensor transform. Converts ndarray to Tensors.
    """
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)
    
class MulTransform:
    """
    Custom Multiplier Transform. Scales the inputs with a value.
    """
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, sample):
        inputs, target = sample
        inputs = inputs * self.factor
        return inputs, target

In [18]:
dataset = WineDataset(transform=None)
first = dataset[0]
feature, labels = first
print(f"Before Applying Transform: {type(feature), type(labels)}")
print(f"Before Applying Transform: {feature, labels}")
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(2)])
dataset = WineDataset(transform=composed)
first = dataset[0]
feature, labels = first
print("================")
print(f"After Applying Transform: {type(feature), type(labels)}")
print(f"After Applying Transform: {feature, labels}")

Before Applying Transform: (<class 'numpy.ndarray'>, <class 'numpy.ndarray'>)
Before Applying Transform: (array([1.423e+01, 1.710e+00, 2.430e+00, 1.560e+01, 1.270e+02, 2.800e+00,
       3.060e+00, 2.800e-01, 2.290e+00, 5.640e+00, 1.040e+00, 3.920e+00,
       1.065e+03], dtype=float32), array([1.], dtype=float32))
After Applying Transform: (<class 'torch.Tensor'>, <class 'torch.Tensor'>)
After Applying Transform: (tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
        6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
        2.1300e+03]), tensor([1.]))
